In [1]:
import os, io
from multiprocessing import Process, Pipe
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.exceptions import *
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac, cmac
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import serialization

def bytes_xor(bytes1, bytes2):
    return b''.join([(a^b).to_bytes(1,byteorder='big') for (a,b) in zip(bytes1, bytes2)])

USED_NONCES = []
BLOCK_SIZE = 32

def gen_nonce():
    nonce = os.urandom(BLOCK_SIZE // 2)
    while nonce in USED_NONCES:
        nonce = os.urandom(BLOCK_SIZE // 2)
    USED_NONCES.append(nonce)
    return nonce

# i is the tweak number
def gen_tweak(i, plaintext_len=0):
    if i > 0:
        nonce = gen_nonce() # Generates nonce with 8 byte
        # Concate between nonce (8 byte) with i (8 byte) [N||i]
        tweak = nonce + int(i).to_bytes(BLOCK_SIZE // 2, byteorder='big') 
        tweak = int.from_bytes(tweak, byteorder='big') # Convert to int
        tweak = tweak >> 1 # Removes last bit
        tweak = tweak << 1 # Adds 0 bit in the last position [N||i||0]
        tweak = tweak.to_bytes(BLOCK_SIZE, byteorder='big')
    else:
        nonce = gen_nonce() # Generates nonce with 8 byte
        # Concate between nonce (8 byte) with plaintext_len (8 byte) [N||b]
        tweak = nonce + plaintext_len.to_bytes(BLOCK_SIZE // 2, byteorder='big')
        # Adds 1 bit in the last position [N||b||1]
        tweak = tweak[:-1] + (tweak[-1] | ord('\x01')).to_bytes(1,byteorder='big')
    
    return tweak

default_algorithm = hashes.SHA256
# seleciona-se um dos vários algorimos implementados na package

def Hash(s):
    digest = hashes.Hash(default_algorithm(),backend=default_backend())
    digest.update(s)
    return digest.finalize()



# geração e derivação de chaves
def gen_priv_key():
    return ec.generate_private_key(
         ec.SECP521R1(), default_backend() #NIST P-521
    )

def gen_pub_key(priv_key):
    return priv_key.public_key().public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

def gen_session_key(priv_key, peer_pub_key):
    shared_key = priv_key.exchange(ec.ECDH(), peer_pub_key)
    session_key = HKDF(
        algorithm=default_algorithm(), 
        length=32,
        salt=None,
        info=b'handshake data',
        backend=default_backend()
    ).derive(shared_key)
    return session_key

def gen_DSA_key():
    return ec.generate_private_key(
        ec.SECP256R1, default_backend() #NIST P-256
    )

In [2]:
message_size = 2**10

def my_mac(key):
    return cmac.CMAC(algorithms.AES(key), backend=default_backend())    

def Emitter(conn):
    
    # Gera o par de chaves Diffie-Hellman
    priv_key = gen_priv_key()
    pub = gen_pub_key(priv_key)
    
    # Gera o par de chaves de autenticação DSA
    DSA_key = gen_DSA_key()
    DSA_pub = gen_pub_key(DSA_key)
    signature = DSA_key.sign(pub+DSA_pub, ec.ECDSA(hashes.SHA256()))
    
    # Efetua-se a troca de chaves públicas
    conn.send((pub, DSA_pub, signature))
    peer_pub_bytes, peer_DSA_bytes, peer_signature = conn.recv() 
    
    peer_pub_key = serialization.load_pem_public_key(
            peer_pub_bytes,
            backend=default_backend()
        )
    peer_DSA_key = serialization.load_pem_public_key(
            peer_DSA_bytes,
            backend=default_backend()
        )
    
    try:
        # Autenticação dos agentes
        peer_DSA_key.verify(peer_signature, peer_pub_bytes+peer_DSA_bytes, ec.ECDSA(hashes.SHA256()))
        
        # Derivação da chave de sessão
        session_key = gen_session_key(priv_key, peer_pub_key)
    
        # Cria um input stream com um número grande de bytes aleatórios
        #    inputs = io.BytesIO(os.urandom(message_size))
        inputs = io.BytesIO(bytes('1'*message_size,'utf-8'))

        # nonce para inicialização do modo CRT: deve ser distinto e aleatório
        nonce  = os.urandom(16)
    
        # Dados associados
        dad     = bytes('Dados associados desta comunicação','utf-8')
    
        # geração do contexto de cifra
        cipher = Cipher(algorithms.AES(session_key), modes.CTR(nonce), 
                        backend=default_backend()).encryptor()

        # escolher um mac e inicializa-lo com os dados associados
        mac = my_mac(session_key)
        mac.update(dad)

        # comunicação e operação de cifra    
        conn.send((nonce)) # envia pela conecção o nonce
        buffer = bytearray(32) # define um buffer para onde vão ser lidos, sucessivamente, os vários blocos do input
    
        tweak_num = 1
        checksum = b''
    
        # lê, cifra e envia sucessivos blocos do input 
        try:     
            while inputs.readinto(buffer):
                if tweak_num == 1:
                    checksum = bytes(buffer)
                else:
                    checksum = bytes_xor(checksum, bytes(buffer))
                tweak = gen_tweak(tweak_num) # gera o tweak para o buffer correspondente
                tweak_num += 1
                conn.send(tweak) # envia o tweak
                ciphertext = cipher.update(bytes_xor(cipher.update(bytes(buffer)), tweak))
                mac.update(ciphertext)
                conn.send((ciphertext, mac.copy().finalize(), None))   

            tweak = gen_tweak(0, 32)
            tag = cipher.update(bytes_xor(cipher.update(checksum), tweak))
            tag = tag[:BLOCK_SIZE//2]
            conn.send(tweak)
            conn.send((cipher.finalize(), mac.finalize(), tag))    # envia a finalização
        except Exception as err:
            print("Erro no emissor: {0}".format(err))

    except InvalidSignature as err:
        print("Erro: Assinatura da chave inválida no emissor: {0}".format(err))
        
    inputs.close()          # fecha a 'input stream'
    conn.close()            # fecha a conecção
        

In [3]:
def Receiver(conn):
    
    # Gera o par de chaves
    priv_key = gen_priv_key()
    pub = gen_pub_key(priv_key)
    
    
    
    # Efetua-se a troca de chaves públicas e gera a chave de sessão
    peer_pub_bytes, peer_DSA_bytes, peer_signature = conn.recv()
    
    try:
        # Autenticação dos agentes
        #peer_DSA_key = ec.EllipticCurvePublicKey.from_encoded_point(ec.SECP256R1(), peer_DSA_bytes)
        peer_DSA_key = serialization.load_pem_public_key(
            peer_DSA_bytes,
            backend=default_backend()
        )
        peer_DSA_key.verify(peer_signature, peer_pub_bytes+peer_DSA_bytes, ec.ECDSA(hashes.SHA256()))
        
        # Gera o par de chaves de autenticação DSA
        DSA_key = gen_DSA_key()
        DSA_pub = gen_pub_key(DSA_key)
        signature = DSA_key.sign(pub+DSA_pub, ec.ECDSA(hashes.SHA256()))
    
        conn.send((pub, DSA_pub, signature))
        
        peer_pub_key = serialization.load_pem_public_key(
            peer_pub_bytes,
            backend=default_backend()
        )
        
        session_key = gen_session_key(priv_key, peer_pub_key)

        # Inicializa um output stream para receber o texto decifrado
        outputs = io.BytesIO()

        # Recuperar a informação de nonce e salt
        nonce = conn.recv()
    
        # Dados associados
        dad     = bytes('Dados associados desta comunicação','utf-8')
    
        # Gerar o contexto de decifragem
        cipher = Cipher(algorithms.AES(session_key), modes.CTR(nonce), 
                        backend=default_backend()).decryptor() 

        # escolher um mac e inicializa-lo com os dados associados
        mac = my_mac(session_key)
        mac.update(dad)
        first = True

        # operar a cifra: ler da conecção um bloco, autenticá-lo, decifrá-lo e escrever o resultado no 'stream' de output
        try:
            while True:    
                try:
                    tweak = conn.recv()
                    buffer, tag, final_tag = conn.recv()
                    ciphertext = bytes(buffer)
                    mac.update(ciphertext)
                    if tag != mac.copy().finalize():
                        raise InvalidSignature("erro no bloco intermédio")
                    plaintext = cipher.update(bytes_xor(cipher.update(ciphertext), tweak))
                    outputs.write(plaintext)
                    if first:
                        checksum = plaintext
                        first = False
                    elif plaintext:
                        checksum = bytes_xor(checksum, plaintext)
                except EOFError:
                    if tag != mac.finalize():
                        raise InvalidSignature("erro na finalização")
                    new_tag = cipher.update(bytes_xor(cipher.update(checksum), tweak))
                    new_tag = new_tag[:BLOCK_SIZE//2]
                    if new_tag != final_tag:
                        raise Exception("Checksum não é válido.")
                    outputs.write(cipher.finalize())
                    break
                except InvalidSignature as err:
                    raise Exception("autenticação do ciphertext ou metadados: {}".format(err))

            print(outputs.getvalue())     # verificar o resultado

        except Exception as err:
            print("Erro no receptor: {0}".format(err))
              
    except InvalidSignature as err:
        print("Erro: Assinatura da chave inválida no emissor: {0}".format(err))
    
    outputs.close()    # fechar 'stream' de output
    conn.close()       # fechar a conecção

In [7]:
class BiConn(object):
    def __init__(self,left,right,timeout=None):
        """
        left : a função que vei ligar ao lado esquerdo do Pipe
        right: a função que vai ligar ao outro lado
        timeout: (opcional) numero de segundos que aguarda pela terminação do processo
        """
        left_end, right_end = Pipe()
        self.le = left_end
        self.timeout=timeout
        self.lproc = Process(target=left, args=(left_end,))       # os processos ligados ao Pipe
        self.rproc = Process(target=right, args=(right_end,))
        self.left  = lambda : left(left_end)                       # as funções ligadas já ao Pipe
        self.right = lambda : right(right_end)

    def auto(self, proc=None):
        if proc == None:             # corre os dois processos independentes
            self.lproc.start()
            self.le.close()
            self.rproc.start()  
            self.lproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:                        # corre só o processo passado como parâmetro
            proc.start(); proc.join()

    def manual(self):   #  corre as duas funções no contexto de um mesmo processo Python
        self.left()
        self.right()

BiConn(Emitter,Receiver,timeout=30).auto()

b'11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111